In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
data = pd.read_csv('edges_ex.txt', sep=" ", header=None, names=['source', 'target'])

In [ ]:
g = nx.from_pandas_edgelist(data, create_using = nx.DiGraph())
g = g.reverse()
print(nx.info(g))
nx.draw(g, with_labels = False)
plt.show

In [ ]:
def triad():
    triads = []
    for u in g:
        u_followers = g.predecessors(u)
        
        u_following = g.successors(u)
        
        
        for w in u_followers:
            w_following = g.successors(w)
            
            for v in w_following:
                l = [x for x in g.successors(u)]
                if v in l:
                    #print(v)
                    tup = (u, v, w)
                    triads.append(tup)
    print(triads)
    print(len(triads))
    return triads
                 
                    

In [ ]:
t = triad()

In [ ]:
def check_ids(t):
  for idx,i in enumerate(t):
    idx = idx+1
    
    u = i[0]
    v = i[1]
    w = i[2]

    try:
      u_user = client.get_user(username = u, user_fields='created_at')
      v_user = client.get_user(username = v, user_fields='created_at')
      w_user = client.get_user(username = w, user_fields='created_at')

    except Exception as e:
      print(e)
      print('User not available')
      t.remove(idx)
  return t
t1 = check_ids(t)

In [ ]:
def timed_triads(t):
  for idx, i in enumerate(t):

    u = i[0]
    v = i[1]
    w = i[2]

    w_id = twitter_id_lookup(w)
    w_followers = client.get_users_following(id = int(w_id))

    u_id = 0
    v_id = 0

    for idy, j in enumerate(w_followers):
      if (u == j.id):
        u_id = idy
      if (v == j.id):
        v_id = idy
  
    if u_id > v_id:
      t.remove(triad_count)

  return t 

t2 = timed_triads(t1)

In [ ]:
def non_rec(v,w):
    w_followers = g.predecessors(w)
    
    if v in [x for x in w_followers]:
        return 0
    return 1

def sim(u,v):
    u_following = g.successors(u)
    v_following = g.successors(v)
    
    u_following = set([x for x in u_following])
    v_following = set([x for x in v_following])
    
    intersection = len(u_following.intersection(v_following))
    union = len(u_following.union(v_following))
    
    similarity = intersection / union
    
    
    
    return similarity 

In [ ]:
def probability_copy(triad):
    non_rec_value = non_rec(triad[1], triad[2])
        
    sim1_val = sim(triad[0], triad[1])
    sim2_val = sim(triad[2], triad[0])
        
    prob = non_rec_value * sim1_val * sim2_val #implicit structure and time ordered
    return prob

In [ ]:
def sum_prob(triad_list):
    ans_list = [] 
    for e in triad_list:
        ans = 0
        for f in triad_list:
            if e[1] == f[1] and e[2] == f[2]:
                ans += probability_copy(f)
        ans_list.append(ans)
    return ans_list      

In [ ]:
ans_list = sum_prob(t)
def prob_list(t, ans_list):
    p_list = []
    for i in range(len(t)):
        p = 0
        if ans_list[i] == 0:
            p = 0
        else:
            p = probability_copy(t[i]) / ans_list[i]
        p_list.append(p)
    return p_list
    

In [ ]:
pro_list = prob_list(t, ans_list)

In [ ]:
def cf_score(u, pro_list):
    sum = 0
    for i in range(len(t)):
        if t[i][0] == u:
            sum += pro_list[i]
    return sum

In [ ]:
cf_score(2, pro_list)

In [ ]:
def imitation_ratio(u):
    cf = cf_score(u, pro_list)
    followers_len = len([x for x in g.predecessors(u)])
    
    friends_len = len([x for x in g.successors(u)])
    
    imitation = cf / (followers_len * friends_len)
    
    return imitation



In [ ]:
imitation_ratio(2)

In [ ]:
def early_adopter_score(u, v):
    u_followers_len = len([x for x in g.predecessors(u)])
    v_followers_len = len([y for y in g.predecessors(v)])
    #print(u_followers_len, v_followers_len)
    early_adopter = imitation_ratio(2) * (u_followers_len / v_followers_len)
    return early_adopter

In [ ]:
early_adopter_score(2,1)

In [ ]:
def future_popularity_score(v):
    sum = 0
    for u in g.predecessors(v):
        sum += early_adopter_score(u,v)
    return sum
        

In [ ]:
future_popularity_score(5)

In [ ]:
def cf_alll():
    cf = []
    P = []
    cf = ['0'] * len(g)
    for w,v in g.edges:
        v_followers = set([x for x in g.predecessors(v)])
        w_friends = set([y for y in g.successors(w)])
        U = v_followers.intersection(w_friends)
        U = list(U)
        
        
        s = 0
        
        for u in U:
            p = probability_copy((u,v,w))
            s += p
            P.append(p)
        
        for i in range(len(U)):
            if s != 0:
                cf[i] = cf_score(U[i])
            else:
                cf[i] = 0
    return cf  

In [ ]:
cf_alll()

In [ ]:
def cf_all():
    cf = []
    P = []
    cf = ['0'] * len(g)
    for w,v in g.edges:
        v_followers = set([x for x in g.predecessors(v)])
        w_friends = set([y for y in g.successors(w)])
        U = v_followers.intersection(w_friends)
        U = list(U)
        
        
        s = 0
        
        for u in U:
            p = probability_copy((u,v,w))
            s += p
            P.append(p)
        
        for i in range(len(U)):
            if s != 0:
                print(P[i], s)
                cf[i] = cf_score(U[i]) + (P[i]/s)
            else:
                cf[i] = 0
    return cf

In [ ]:
cf_all()